## Introduction:

MLP tutorial .. 

https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

Up next:

ANN tensorflow .. 

https://www.jessicayung.com/explaining-tensorflow-code-for-a-multilayer-perceptron/

https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py

https://www.analyticsvidhya.com/blog/2016/10/an-introduction-to-implementing-neural-networks-using-tensorflow/

In [1]:
import numpy as np
import pandas as pd 
from random import random
from csv import reader
from math import exp
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_table('seeds_dataset.csv', delim_whitespace=True, names=('1', '2', '3', '4', '5', '6', '7', '8'))

In [3]:
df.head()

,1,2,3,4,5,6,7,8
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [4]:
input_ = df.iloc[:,0:7]
output_ = df.iloc[:,-1]

In [5]:
## initialize the network .. 

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    
    network.append(hidden_layer) #append to generate single matrix for the whole network 
    
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    
    network.append(output_layer)
    
    return network

In [6]:
n_inputs = 7
n_hidden = 12
n_outputs = 3

network = initialize_network(n_inputs, n_hidden, n_outputs)
np.shape(network) #two layers .. from I => H, H => O

(2,)

In [8]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in xrange(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [9]:
## activation function 

# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [167]:
# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row #[:-1]
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
            inputs = new_inputs
    return inputs

In [168]:
# test forward propagation
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
        [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row = [1, 0]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7210355736949032]


## Backpropagating the errors:


In [156]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

In [157]:
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [158]:
# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
        [{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]


In [159]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

In [160]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
#             expected = [0 for i in range(n_outputs)]
#             expected[row[-1]] = 1
#             sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
#             backward_propagate_error(network, expected)
#             update_weights(network, row, l_rate)
#         print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [161]:
## testing the whole algorithm .. 

dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

n_inputs = len(dataset[0])-1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs) #2x2

train_network(network, dataset, 0.5, 20, n_outputs)

np.shape(dataset)

# for layer in network:
#     print(layer)

TypeError: can't multiply sequence by non-int of type 'float'